In [ ]:
# Carga funciones desde graf2.ipynb
# %run -i graf2.ipynb


### SELENIUN 

In [ ]:
!pip install requests beautifulsoup4 lxml
!pip install webdriver-manager selenium
!pip install apify-client
!pip install seleniumbase
!pip install undetected-chromedriver
!pip install -U selenium webdriver-manager bs4

In [ ]:
import time
import random
import undetected_chromedriver as uc

from seleniumbase import Driver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains




In [ ]:
# ---------- Utilidades "humanas" ----------
def human_pause(a=0.4, b=1.2):
    time.sleep(random.uniform(a, b))
def gentle_scroll(driver, steps=5, delta=600):
    for _ in range(steps):
        driver.execute_script(f"window.scrollBy(0, {random.randint(int(delta*0.7), int(delta*1.2))});")
        human_pause(0.3, 0.8)
def move_mouse_random(actions, element):
    # mueve el ratón al elemento con pequeñas pausas
    actions.move_to_element(element).pause(random.uniform(0.2, 0.6)).perform()

# ---------- Utilidades varias ----------
def cookies_acept(driver):
    # --- Cookies: intenta varios selectores comunes en la web ---
    print("Clic aceptar cookies (si aparece)...")
    for sel in [
    # "#onetrust-accept-btn-handler",      # banner OneTrust (el más común)
        "#cookiesAcceptAll",                # el que usabas
    # "button[aria-label='Aceptar todas']",
    ]:
        try:
            btn = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.CSS_SELECTOR, sel)))
            btn.click()
            break
        except TimeoutException:
            pass

In [ ]:
def css_corto(el):
    # Construye un CSS corto tag#id o tag[attr=val]
    tag = (el.tag_name or "div").lower()
    el_id = el.get_attribute("id")
    if el_id:
        return f"{tag}#{el_id}"
    # atributos útiles
    for attr in ["data-testid", "data-test", "aria-label", "name", "title", "alt", "href"]:
        val = el.get_attribute(attr)
        if val:
            # escapado simple de comillas
            val = val.replace('"', '\\"')
            return f'{tag}[{attr}="{val}"]'
    # si tiene clases "legibles"
    cls = el.get_attribute("class") or ""
    clases = [c for c in cls.split() if c and len(c) < 24 and not any(ch.isdigit() for ch in c)]
    if clases:
        return f"{tag}." + ".".join(clases[:2])
    return tag

def listar_clickables(driver, limit=40):
    # Busca anchors, botones y roles similares
    js = """
    const sels = Array.from(document.querySelectorAll(
        'a, button, [role="button"], input[type="button"], input[type="submit"]'
    ));
    return sels.slice(0, arguments[0]);
    """
    elems = driver.execute_script(js, limit)
    out = []
    for el in elems:
        out.append(css_corto(el))
    # Quita duplicados conservando orden
    vistos, unicos = set(), []
    for s in out:
        if s not in vistos:
            vistos.add(s)
            unicos.append(s)
    return unicos

from selenium.common.exceptions import NoSuchElementException, JavascriptException, TimeoutException
import time

# --- Supón que ya tienes candidatos = listar_clickables(driver, limit=60) ---

def mostrar_css_selector_validos(driver, candidatos, pausa=0.4):
    print("\n🔎 Verificando elementos visibles en la página...\n")
    validos = []
    for s in candidatos:
        try:
            # Intenta encontrar el elemento
            elem = driver.find_element(By.CSS_SELECTOR, s)
            if not elem.is_displayed():
                continue  # si está oculto, lo saltamos
            # Resalta el elemento (SeleniumBase)
            driver.highlight(s)
            print(f"✅ → {s}")
            validos.append(s)
            time.sleep(pausa)
        except (NoSuchElementException, JavascriptException, TimeoutException):
            # Elemento no existe o no se puede acceder
            continue
        except Exception as e:
            # Cualquier otro error inesperado
            print(f"⚠️ Error en {s}: {e}")
            continue
    print(f"\n✅ Total elementos válidos mostrados: {len(validos)}")
    return validos



def get_elements_to_click(driver, product_selectors=None, timeout=10):
    """
    Espera y devuelve una lista con todos los elementos clicables visibles
    de una página dinámica (por ejemplo, resultados de productos).

    Params:
        driver : WebDriver/SeleniumBase driver.
        product_selectors : lista de selectores CSS a probar.
        timeout : tiempo máximo de espera (segundos).

    Returns:
        list[WebElement] -> todos los elementos visibles y clicables encontrados.
    """

    if product_selectors is None:
        product_selectors = [
            "div.product-card"
           # "div.product-card a",            # enlace dentro de la tarjeta
           # "article.product-card a",        # variante
           # "li[data-product] a",            # fallback
           # "a.product-card",                # si la tarjeta es un enlace directo
           # "section.products a",            # extra genérico
        ]

    wait = WebDriverWait(driver, timeout)
    clicables = []

    print("🔍 Esperando a que se carguen los resultados...")

    for css in product_selectors:
        try:
            # Esperar a que haya al menos uno visible
            wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, css)))

            # Buscar todos los elementos con ese selector
            elems = driver.find_elements(By.CSS_SELECTOR, css)

            # Filtrar los visibles y habilitados
            visibles = [el for el in elems if el.is_displayed() and el.is_enabled()]

            if visibles:
                clicables.extend(visibles)
                print(f"✅ Encontrados {len(visibles)} elementos con: {css}")

        except TimeoutException:
            print(f"⏳ No se encontraron elementos visibles con {css}")
            continue
        except StaleElementReferenceException:
            print(f"⚠️ DOM cambió durante la búsqueda ({css}). Reintentando...")
            continue
        except Exception as e:
            print(f"⚠️ Error con selector {css}: {e}")
            continue

    print(f"\n🔹 Total elementos clicables detectados: {len(clicables)}")
    return clicables


def get_product_atrubutes (driver, elem_i):
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", elem_i)
        human_pause(0.9, 2.0)
        elem_i.click()

        codigo = driver.find_element(By.CSS_SELECTOR, "#pdp-id").get_attribute("textContent").strip()
        precio = driver.find_element(By.CSS_SELECTOR, "#pdp-price-current-integer").get_attribute("textContent").strip()
        nombre = driver.find_element(By.CSS_SELECTOR, "h1").text.strip()
        human_pause(0.9, 2.0)
        driver.back()
        human_pause(0.9, 2.0)
        a_up = elem_i.find_element(By.XPATH, "./ancestor::a[@href][1]")
        href = a_up.get_attribute("href")
        datos = {
                "nombre": nombre,
                "codigo": codigo,
                "precio": precio,
                "href": href
            }

        human_pause(0.9, 2.0)
        return datos 



In [ ]:
# driver.open("https://frikidelto.com/")
# driver.open("https://motos.coches.net/")
# driver.open("https://www.linkedin.com/feed/")

In [ ]:
# driver = Driver(wire=True)
# driver = Driver(uc=True)
driver = Driver(browser="chrome")
#driver = Driver(browser="edge")
actions = ActionChains(driver)
wait = WebDriverWait(driver, 12)

print("Cargando página...")
# driver.open("https://www.pccomponentes.com/")
cookies_acept(driver)

# --- Buscar término ---
TXT_SEARCH = "portatil microsoft"
print("Activando caja de búsqueda:{TXT_SEARCH}")
search = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#search")))
search.click()

print("Introduciendo texto...")
search.clear()
search.send_keys(TXT_SEARCH)
search.send_keys(Keys.ENTER)

css = "div.product-card"
elems = get_elements_to_click(driver,product_selectors=[css])
if 0:
    productos = []
    for i in range(1,5):
        print(f"\n➡️ Procesando producto #{i} ...")
        elem_i = elems[i]
        datos=get_product_atrubutes (driver, elem_i)
        productos.append(datos)   
    print("\n✅ Datos obtenidos:")
    for p in productos:
        print(p)

In [ ]:
# --- Ejemplo de uso ---
candidatos = listar_clickables(driver, limit=60)
validos = mostrar_css_selector_validos(driver, candidatos)


In [ ]:
s='button[aria-label="Search"]'
driver.highlight(s)

In [ ]:
e = driver.find_element(s)
e.click()

In [ ]:
URL = "https://motos.coches.net/segunda-mano/?st=2&MakeIds%5B0%5D=7&Models%5B0%5D=R%20nineT&pg=3"

In [ ]:
display(Markdown("#### 🚀 Ejemplo Rápido"))
display(Markdown("#### 🎮 Inicializar Consola Virtual"))


In [ ]:
# https://console.apify.com/actors/losKmlMFXD1mjjbMp/runs/xvlIvprLkojVPODWl#output

In [ ]:
import os
from apify_client import ApifyClient

exe = 0
YOUR_API_TOKEN = os.getenv('APIFY_API_TOKEN') or ''
# Initialize the ApifyClient with your API token
client = ApifyClient(YOUR_API_TOKEN)

# Prepare the Actor input
run_input = {
    "tipo": "segunda-mano",
    "marca": ["BMW"],
    "modelo": ["R 12 nineT"],
    "maxItems": 200
}

# Run the Actor and wait for it to finish
if exe:
    run = client.actor("losKmlMFXD1mjjbMp").call(run_input=run_input)

    result = []
    # Fetch and print Actor results from the run's dataset (if there are any)
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        print(item)
        result.append(item)

    # Ahora 'result' contiene todos los elementos como objetos JSON
    print(f"Se guardaron {len(result)} elementos en la variable 'result'")
    


#### 1.2. Guardado de datos en archivo json

In [ ]:
import json

# Guardar en archivo JSON
with open('data/motos/motos_bmw.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print("Datos guardados en 'motos_bmw.json'") 

#### 1.3. Lectura de los datos 

In [2]:
import pandas as pd
import numpy as np
from src.load_data import get_items_json

#carga de datos json
result = get_items_json ()

Cargados 1 archivo(s) JSON
Extaidos 1 contenidos JSON
Extaidos 2 contenidos HTML
Extaidos  2 contenidos ITMEMS
Extaidos  24 contenidos ITMEMS


In [4]:

# Convertir a DataFrame
df = pd.DataFrame(result)

# Mostrar las primeras filas
df.head()


,id,url,title,km,price,year,imgUrl,provinceId,hp
0,8762366,https://motos.coches.net/ocasion/bmw/r_ninet/s...,BMW R nineT Scrambler,8000,9995,2018,https://a.mcdn.es/mnet/2025/08/26/8762366/3237...,20,110.0
1,8760435,https://motos.coches.net/ocasion/bmw/r_ninet//...,BMW R nineT /5,12960,11990,2019,https://a.mcdn.es/mnet/2025/08/23/8760435/3235...,28,NaN
2,8760378,https://motos.coches.net/ocasion/bmw/r_ninet/2...,BMW R nineT,22500,9500,2014,https://a.mcdn.es/mnet/2025/08/23/8760378/3235...,30,NaN
3,8759793,https://motos.coches.net/ocasion/bmw/r_ninet/2...,BMW R nineT,9500,10500,2014,https://a.mcdn.es/mnet/2025/08/22/8759793/3235...,28,NaN
4,8759386,https://motos.coches.net/ocasion/bmw/r_ninet/2...,BMW R nineT,20000,8500,2017,https://a.mcdn.es/mnet/2025/08/21/8759386/3234...,7,NaN


In [ ]:
precios = [moto.get('price', 0) for moto in result if moto.get('price')]
if precios:
    print(f"Precio promedio: {sum(precios) / len(precios):.2f} €")
    print(f"Precio más bajo: {min(precios)} €")
    print(f"Precio más alto: {max(precios)} €")

In [ ]:
# Filtrar motos por precio
motos_baratas = [moto for moto in result if moto.get('price', 0) < 10000]
motos_caras = [moto for moto in result if moto.get('price', 0) >= 10000]

print(f"Motos por debajo de 10,000€: {len(motos_baratas)}")
print(f"Motos por encima de 10,000€: {len(motos_caras)}")

In [ ]:
CAMPO = 'title'
dg = df.groupby(CAMPO).size().reset_index(name='num_registros')
dg = dg.sort_values(by='num_registros', ascending=False)
dg.head()

In [5]:

ATRIBUTO = 'hp'

# Agrupar por año de matriculación y contar registros
dg = (
    df.groupby(ATRIBUTO)
      .size()
      .reset_index(name='num_registros')
      .sort_values(by=ATRIBUTO,ascending=False)
)
dg.head()

,hp,num_registros
0,110.0,11


In [6]:
# Asegurarse de que la columna 'price' sea numérica
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Definir los límites de los intervalos de precio (0 a 48000, cada 6000)
bins = np.arange(0, 48001, 6000)

# Crear etiquetas como "0-6000", "6000-12000", ...
labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins)-1)]

# Asignar cada precio a su intervalo
df['rango_precio'] = pd.cut(df['price'], bins=bins, labels=labels, right=False)

ATRIBUTO = 'rango_precio'
dg = (
    df.groupby(ATRIBUTO)
      .size()
      .reset_index(name='num_registros')
      .sort_values(by=ATRIBUTO,ascending=False)
)
dg


C:\Users\jaime.lopez\AppData\Local\Temp\ipykernel_8072\3336554779.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(ATRIBUTO)


,rango_precio,num_registros
7,42000-48000,0
6,36000-42000,0
5,30000-36000,0
4,24000-30000,0
3,18000-24000,2
2,12000-18000,6
1,6000-12000,16
0,0-6000,0
